In [1]:
%%capture

################################################
### This code download the WoodScape Dataset ###
################################################

# GitHub: https://github.com/valeoai/WoodScape.git
# Google Drive: https://drive.google.com/drive/folders/1X5JOMEfVlaXfdNy24P8VA-jMs0yzf_HR

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# clone the the github repo of the WoodScape dataset
!git clone https://github.com/valeoai/WoodScape.git

# semantic annotation
file_id = '1CBwi0fpDE2G99hHiINTI-AXlOMQSUnb-'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('semantic_annotations.zip')
# instance annotation
file_id = '1k9q1k8rh6hghSPFdmxmkZUd_Mip6pt6F'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('instance_annotations.zip')
# rgb images (train data)
file_id = '1xQ5J4huNmyK9WPoipHTnuZ7lw_J0xhvL'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('rgb_images.zip')
# rgb images (test data)
file_id = '11C9QM3a5Mnu3SO4iJcoQcQzBKMMTU7HL'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('rgb_images(test_set).zip')

# unzip files
!unzip /content/semantic_annotations.zip
!unzip /content/instance_annotations.zip
!unzip /content/rgb_images.zip
!unzip /content/rgb_images\(test_set\).zip

# remove .zip file
!rm /content/semantic_annotations.zip
!rm /content/instance_annotations.zip
!rm /content/rgb_images.zip
!rm /content/rgb_images\(test_set\).zip

# move images' folders into WoodScape/data dir to better organize
!mv /content/instance_annotations /content/WoodScape/data
!mv /content/semantic_annotations /content/WoodScape/data
!mv /content/rgb_images /content/WoodScape/data
!mv /content/rgb_images\(test_set\) /content/WoodScape/data

# generate the semantic segmentation annotations from json instance annotations
!python /content/WoodScape/scripts/semantic_map_generator.py --src_path "/content/WoodScape/data/instance_annotations/" --dst_path "/content/WoodScape/data/semantic_annotations" --semantic_class_mapping "/content/WoodScape/scripts/configs/semantic_mapping_9_classes.json" --instance_class_mapping "/content/WoodScape/scripts/mappers/class_names.json"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
from shutil import copyfile

img_dir='/content/WoodScape/data/rgb_images'
annotation_dir='/content/WoodScape/data/semantic_annotations/gtLabels'

images = sorted([os.path.join(img_dir, file) for file in os.listdir(img_dir) if file.endswith('.png')])

for idx in range(5000, len(images)):
    img_path = images[idx]
    annotation_path = os.path.join(annotation_dir, os.path.basename(img_path))
    img_dest = os.path.join("/content/drive/MyDrive/vision_project/dataset/woodscape/data/images", os.path.basename(img_path))
    annotation_dest = os.path.join("/content/drive/MyDrive/vision_project/dataset/woodscape/data/semantic_annotations", os.path.basename(img_path))
    copyfile(img_path, img_dest)
    copyfile(annotation_path, annotation_dest)

In [ ]:
%%capture

############################################
### This code exrtact 3k random examples ###
############################################

import os
import shutil
import numpy as np

np.random.seed(3)

os.mkdir("/content/3k_examples")
os.mkdir("/content/3k_examples/images")
os.mkdir("/content/3k_examples/semantic_annotations")

img_dir='/content/WoodScape/data/rgb_images'
annotation_dir='/content/WoodScape/data/semantic_annotations/gtLabels'

N_EXAMPLES = 3000

idx = np.random.shuffle(np.arange(len(img_dir)))[:N_EXAMPLES]

for i in idx:
    img = os.listdir(img_dir)[i]
    src = os.path.join(img_dir, img)
    dst = os.path.join("/content/3k_examples/images/", img)
    shutil.copyfile(src, dst)

for i in idx:
    img = os.listdir(annotation_dir)[i]
    src = os.path.join(annotation_dir, img)
    dst = os.path.join("/content/3k_examples/semantic_annotations/", img)
    shutil.copyfile(src, dst)

!zip -r /content/3k_examples.zip /content/3k_examples